Arthur Catarino de Oliveira                               
Guilherme Fidélis Freire                                           
Mateus Poddis Correa

Este notebook resolve um problema de alocação de recursos para construção de residências, maximizando a arrecadação de impostos dentro de restrições de área e financiamento.

In [ ]:
!pip install pulp
import pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 80.2 MB/s eta 0:00:00


## Dados

- `area[i]`: área ocupada pelo tipo de residência *i*.  
- `custo[i]`: custo de construção da residência *i*.  
- `imposto[i]`: imposto arrecadado por cada residência do tipo *i*.  
- `financiamento`: valor máximo de financiamento disponível.  


In [ ]:
area  = [0.18, 0.28, 0.4, 0.5] #area para construir cada tipo de residência
custo = [50000, 70000, 130000, 160000] # custo para contruir cada residência
imposto = [1000, 1900, 2700, 3400] # imposto arrecadado por cada residência
financiamento = 150000000
n = len(area)

## Variáveis de decisão

- `x[i]`: quantidade de residências do tipo *i* a serem construídas.  
- `D`: Numero de casas a serem demolidas

In [ ]:
modelo = pulp.LpProblem('ConstrucaoDeResidências', pulp.LpMaximize)

x = pulp.LpVariable.dicts("x", range(n), lowBound=0, cat=pulp.LpInteger)
D = pulp.LpVariable("D", lowBound=0, cat=pulp.LpInteger)

## Função Objetivo

Maximizar a arrecadação de impostos:

$$
Z \;=\; \sum_{i=0}^{n-1} x_i \cdot imposto_i
$$

In [ ]:
modelo += pulp.lpSum([x[i] * imposto[i] for i in range(n)]), "ArrecadacaoTotal"

## Restrições

1. **Área disponível em função das casas a serem demolidas**  
   $$
   \sum_{i=0}^{n-1} x_i \cdot area_i \le 0.25 \cdot 0.85 \cdot D
   $$

2. **Custo total de construção **  
   $$
   \sum_{i=0}^{n-1} x_i \cdot custo_i + 2000 \cdot D \le financiamento
   $$

3. **Proporção mínima de tipos de residências**

   - Tipo 0 ocupa pelo menos 20% da área construída:  
     $$x_0 \cdot area_0 \ge 0.2 \cdot \sum_{i=0}^{n-1} x_i \cdot area_i$$

   - Tipo 1 ocupa pelo menos 10% da área construída:  
     $$x_1 \cdot area_1 \ge 0.1 \cdot \sum_{i=0}^{n-1} x_i \cdot area_i$$

   - Tipos 2 e 3 juntos ocupam pelo menos 25% da área construída:  
     $$(x_2 \cdot area_2 + x_3 \cdot area_3) \ge 0.25 \cdot \sum_{i=0}^{n-1} x_i \cdot area_i$$

4. **Limite de casas a serem demolidas**  
   $$D \le 300$$

In [ ]:
modelo += pulp.lpSum([x[i] * area[i] for i in range(n)]) <= D * 0.25 * 0.85
modelo += pulp.lpSum([x[i] * custo[i] for i in range(n)]) + (D * 2000) <= financiamento
modelo += x[0] * area[0] >= 0.2 * pulp.lpSum([x[i] * area[i] for i in range(n)])
modelo += x[1] * area[1] >= 0.1 * pulp.lpSum([x[i] * area[i] for i in range(n)])
modelo += x[2] * area[2] + x[3] * area[3] >= 0.2 * pulp.lpSum([x[i] * area[i] for i in range(n)])
modelo += D <= 300

Solução do problema:

In [ ]:
modelo.solve()

print("Status:", pulp.LpStatus[modelo.status])
print("Simples    =", int(round(x[0].value())))
print("Duplos     =", int(round(x[1].value())))
print("Triplos    =", int(round(x[2].value())))
print("Quádruplos =", int(round(x[3].value())))
print("Casas demolidas   =", int(round(D.value())))
print("Arrecadação Total =", round(pulp.value(modelo.objective), 2))

Status: Optimal
Simples    = 71
Duplos     = 27
Triplos    = 1
Quádruplos = 86
Casas demolidas   = 300
Arrecadação Total = 417400.0
